<a href="https://colab.research.google.com/github/xdpikaboo/CSGY_6033_Final_Project/blob/main/LLM_Powered_Dataset_Description_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import os
import re
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
from google.colab import files
from typing import Dict, Any, List, Tuple
from openai import OpenAI
import getpass

# DOWNLOAD TEST DATASET

In [2]:
# ---------- 1) Load CSV ----------
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")
csv_name = list(uploaded.keys())[0]
csv_bytes = uploaded[csv_name]
df = pd.read_csv(io.BytesIO(csv_bytes), low_memory=False)

Saving NYC_Parks_Monuments.csv to NYC_Parks_Monuments (1).csv


# ENTER OpenAI API KEY

In [3]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


#DATA PROFILER

In [4]:
"""
=== Helper functions ===
This cell defines small utility functions used by the profiler:
- _try_to_datetime: safely attempts to convert a column to datetime
- _safe_quantiles: compute quantiles for numeric columns (handles errors gracefully)
- _histogram: produce histogram bin centers + counts for numeric data
- _top_values: returns the most frequent categorical values + percentages
- _text_stats: computes text length statistics for text-like columns
"""

def _try_to_datetime(series: pd.Series) -> Tuple[pd.Series, bool]:
    if pd.api.types.is_datetime64_any_dtype(series):
        return series, True
    try:
        s = pd.to_datetime(series, errors="raise", infer_datetime_format=True)
        return s, True
    except Exception:
        return series, False

def _safe_quantiles(s: pd.Series, qs=(0.05, 0.25, 0.5, 0.75, 0.95)) -> Dict[str, float]:
    try:
        q = s.quantile(qs)
        return {f"q{int(p*100)}": float(v) for p, v in q.items() if pd.notna(v)}
    except Exception:
        return {}

def _histogram(s: pd.Series, bins: int = 10) -> Dict[str, Any]:
    arr = pd.to_numeric(s, errors="coerce").dropna().to_numpy()
    if arr.size == 0 or np.nanmax(arr) == np.nanmin(arr):
        return {"bins": [], "counts": []}
    counts, edges = np.histogram(arr, bins=bins)
    centers = (edges[:-1] + edges[1:]) / 2.0
    return {"bins": centers.tolist(), "counts": counts.astype(int).tolist()}

def _top_values(s: pd.Series, k: int = 10) -> List[Dict[str, Any]]:
    vc = s.value_counts(dropna=True).head(k)
    total = int(s.notna().sum())
    return [{"value": str(v), "count": int(c), "pct": float(c/total) if total else 0.0}
            for v, c in vc.items()]

def _text_stats(s: pd.Series) -> Dict[str, Any]:
    s2 = s.dropna().astype(str)
    if s2.empty:
        return {"avg_length": None, "q25_length": None, "q50_length": None, "q75_length": None}
    lens = s2.str.len()
    return {
        "avg_length": float(lens.mean()),
        "q25_length": float(lens.quantile(0.25)),
        "q50_length": float(lens.quantile(0.50)),
        "q75_length": float(lens.quantile(0.75)),
    }


In [5]:
"""
=== Semantic type inference ===
This cell defines `_semantic_type`, which labels each column as:
  boolean, integer, float, datetime, categorical, or text.
The function uses dtype checks + heuristics (e.g., 80% successful datetime parsing).
The output is more meaningful than raw pandas dtypes.
"""

def _semantic_type(series: pd.Series) -> str:
    if pd.api.types.is_bool_dtype(series): return "boolean"
    if pd.api.types.is_numeric_dtype(series):
        return "integer" if pd.api.types.is_integer_dtype(series) else "float"
    if pd.api.types.is_datetime64_any_dtype(series): return "datetime"

    # try a light datetime guess on a small sample
    sample = series.dropna().astype(str).head(50)
    parsed = pd.to_datetime(sample, errors="coerce", infer_datetime_format=True)
    if parsed.notna().mean() >= 0.8:
        return "datetime"

    nunique = series.nunique(dropna=True)
    ratio = nunique / max(len(series), 1)
    return "categorical" if nunique <= 50 and ratio <= 0.5 else "text"

In [6]:
"""
=== Geospatial + Key Detection ===
This cell provides:
  - _is_lat / _is_lon: detect latitude/longitude columns
  - _geospatial_hints: determine whether dataset has lat/lon pair
  - _primary_key_candidates: detect columns that are unique + non-null
Used later to enrich dataset-level metadata.
"""

_LAT = {"lat", "latitude"}; _LON = {"lon", "lng", "long", "longitude"}

def _is_lat(name: str, s: pd.Series) -> bool:
    name = name.lower()
    if name in _LAT and pd.api.types.is_numeric_dtype(s): return True
    return pd.api.types.is_numeric_dtype(s) and s.min(skipna=True) >= -90 and s.max(skipna=True) <= 90

def _is_lon(name: str, s: pd.Series) -> bool:
    name = name.lower()
    if name in _LON and pd.api.types.is_numeric_dtype(s): return True
    return pd.api.types.is_numeric_dtype(s) and s.min(skipna=True) >= -180 and s.max(skipna=True) <= 180

def _geospatial_hints(df: pd.DataFrame) -> Dict[str, Any]:
    lat = next((c for c in df.columns if _is_lat(c, df[c])), None)
    lon = next((c for c in df.columns if _is_lon(c, df[c]) and c != lat), None)
    return {"lat_column": lat, "lon_column": lon, "has_latlon_pair": lat is not None and lon is not None}

def _primary_key_candidates(df: pd.DataFrame) -> List[str]:
    return [c for c in df.columns if df[c].isna().mean() == 0 and df[c].is_unique]

In [7]:
"""
=== Full Data Profiler ===
This cell defines `profile_dataframe(df)` which:
  - Profiles each column (stats, missingness, semantic type, distributions, etc.)
  - Computes dataset-level metadata (row count, column count, pk candidates, geospatial hints)
  - Returns everything as a structured JSON-friendly dictionary.
This is the main profiling engine.
"""
def profile_dataframe(
    df: pd.DataFrame,
    *,
    sample_rows: int = 1_000_000,
    hist_bins: int = 10,
    topk_categorical: int = 10,
) -> Dict[str, Any]:
    # downsample (row-wise) for very large inputs
    if len(df) > sample_rows:
        df = df.sample(sample_rows, random_state=42)

    n_rows, n_cols = df.shape
    mem_bytes = int(df.memory_usage(deep=True).sum())

    column_summaries = {}
    type_counts: Dict[str, int] = {}

    for c in df.columns:
        s = df[c]
        sem = _semantic_type(s)
        type_counts[sem] = type_counts.get(sem, 0) + 1

        non_null = int(s.notna().sum())
        null_frac = float(s.isna().mean())
        nunique = int(s.nunique(dropna=True))
        is_unique = s.is_unique and null_frac == 0.0
        is_const = nunique == 1

        info: Dict[str, Any] = {
            "dtype": str(s.dtype),
            "semantic_type": sem,
            "non_null_count": non_null,
            "null_fraction": null_frac,
            "n_unique": nunique,
            "is_unique": bool(is_unique),
            "is_constant": bool(is_const),
            "example_values": [str(v) for v in s.dropna().unique()[:5]],
        }

        if sem in {"integer", "float"}:
            s_num = pd.to_numeric(s, errors="coerce")
            info.update({
                "min": None if s_num.dropna().empty else float(s_num.min()),
                "max": None if s_num.dropna().empty else float(s_num.max()),
                "mean": None if s_num.dropna().empty else float(s_num.mean()),
                "std": None if s_num.dropna().empty else float(s_num.std(ddof=1)),
                "quantiles": _safe_quantiles(s_num),
                "histogram": _histogram(s_num, bins=hist_bins),
            })

        elif sem == "datetime":
            s_dt, ok = _try_to_datetime(s)
            if ok and not s_dt.dropna().empty:
                info.update({
                    "min": s_dt.min(),
                    "max": s_dt.max(),
                    "is_monotonic_increasing": bool(s_dt.is_monotonic_increasing),
                    "is_monotonic_decreasing": bool(s_dt[::-1].is_monotonic_increasing),
                })

        elif sem == "categorical":
            info["top_values"] = _top_values(s.astype("string"), k=topk_categorical)

        elif sem == "text":
            info["text_stats"] = _text_stats(s)

        column_summaries[c] = info

    # dataset-level
    geohints = _geospatial_hints(df)
    pk = _primary_key_candidates(df)

    time_min, time_max = None, None
    for c, info in column_summaries.items():
        if info["semantic_type"] == "datetime" and info.get("min") is not None:
            tmin, tmax = info["min"], info["max"]
            time_min = tmin if time_min is None or (tmin is not None and tmin < time_min) else time_min
            time_max = tmax if time_max is None or (tmax is not None and tmax > time_max) else time_max

    return {
        "num_rows": int(n_rows),
        "num_columns": int(n_cols),
        "memory_usage_bytes": mem_bytes,
        "duplicate_row_fraction": float((n_rows - len(df.drop_duplicates())) / n_rows) if n_rows else 0.0,
        "type_counts": type_counts,
        "time_coverage": {"min": time_min, "max": time_max},
        "geospatial_hints": geohints,
        "primary_key_candidates": pk,
        "columns": column_summaries,
    }

# DATA SIGNAL

In [8]:
"""
=== Build Data Signals ===
This cell defines `build_data_signals(profile)` which:
  • Takes the raw profiler output
  • Aggregates it into simplified dataset-level signals
    (semantic type counts, missingness fraction, PKs, lat/lon presence, etc.)
  • Produces a compact summary dictionary intended for LLM input
    BEFORE generating natural-language summaries.
"""
def build_data_signals(profile: Dict[str, Any]) -> Dict[str, Any]:
    """
    Turn the raw profiler output into higher-level 'data signals'
    that are easier for an LLM to consume.
    """
    columns = profile.get("columns", {})
    dataset = profile.get("dataset", {})

    # Handle both naming variants just in case
    n_rows = dataset.get("n_rows") or dataset.get("num_rows")
    n_cols = dataset.get("n_cols") or dataset.get("num_columns")

    # ---- Dataset-level signals ----
    # Count semantic types
    type_counts: Dict[str, int] = {}
    total_missing = 0
    for name, info in columns.items():
        sem = info.get("semantic_type")
        if sem:
            type_counts[sem] = type_counts.get(sem, 0) + 1
        total_missing += int(info.get("n_missing", 0))

    total_cells = n_rows * n_cols if (n_rows is not None and n_cols is not None) else None
    overall_missing_fraction = (
        float(total_missing / total_cells) if total_cells and total_cells > 0 else None
    )

    # geospatial + primary keys may or may not exist depending on your profiler version
    geospatial = dataset.get("geospatial") or dataset.get("geospatial_hints") or {}
    pk_candidates = dataset.get("primary_keys") or dataset.get("primary_key_candidates") or []

    dataset_signals: Dict[str, Any] = {
        "n_rows": n_rows,
        "n_cols": n_cols,
        "type_counts": type_counts,                      # how many numeric / text / datetime / etc
        "overall_missing_fraction": overall_missing_fraction,
        "has_geospatial": bool(geospatial.get("has_latlon_pair")),
        "lat_column": geospatial.get("lat_column"),
        "lon_column": geospatial.get("lon_column"),
        "primary_key_candidates": pk_candidates,
    }

    # ---- Per-column signals ----
    column_signals: List[Dict[str, Any]] = []

    for name, info in columns.items():
        sem = info.get("semantic_type")
        n_missing = int(info.get("n_missing", 0))
        nunique = int(info.get("nunique") or info.get("n_unique", 0))

        missing_fraction = float(n_missing / n_rows) if n_rows and n_rows > 0 else 0.0

        # example values from top_values (if present)
        top_vals = info.get("top_values") or []
        example_values = [tv.get("value") for tv in top_vals[:3]] if isinstance(top_vals, list) else []

        # use 5% and 95% quantiles as an approximate range, if numeric
        quantiles = info.get("quantiles") or {}
        approx_min = quantiles.get("0.05")
        approx_max = quantiles.get("0.95")

        # simple heuristics for "id-like" and "binary" columns
        is_id_like = bool(n_rows and nunique == n_rows and missing_fraction == 0.0)
        is_binary = bool(
            nunique == 2
            and sem in {"integer", "float", "boolean", "categorical"}
        )

        column_signals.append(
            {
                "name": name,
                "semantic_type": sem,
                "missing_fraction": missing_fraction,
                "n_unique": nunique,
                "is_id_like": is_id_like,           # good candidate for primary key / identifier
                "is_binary": is_binary,             # good for yes/no flags
                "approx_min": approx_min,           # for numeric/datetime: rough range
                "approx_max": approx_max,
                "example_values": example_values,   # a few sample values as strings
            }
        )

    return {
        "dataset_signals": dataset_signals,
        "column_signals": column_signals,
    }

# SUMMARY GENERATOR (LLM)

In [9]:
def summarize_dataset_with_llm(
    profile: Dict[str, Any],
    signals: Dict[str, Any],
    model: str = "gpt-4o-mini"
) -> str:

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # ---- Convert dict → JSON ----
    dataset_summary_json = json.dumps(
        {
            "dataset_signals": profile,
            "column_signals": signals.get("column_signals", [])
        },
        indent=2,
        default=str
    )

    # ---- Dataset-card prompt ----
    user_prompt = f"""
        You are a data documentation assistant.

        A machine-generated content summary of a tabular dataset in this JSON format:

        {dataset_summary_json}

        The JSON has keys such as:
        - "dataset_signals" (n_rows, n_cols, type_counts, overall_missing_fraction, has_geospatial, etc.)
        - "column_signals": a list of columns with fields like name, semantic_type, missing_fraction, n_unique, example_values, etc.

        Your job is to write a VERY concise “dataset card” style description that:
        - Uses the JSON only to infer what the data contains and how it can be used.
        - Uses the user fields to tailor the framing to their interest.
        - Does NOT invent specific values (dates, row counts, monetary amounts, percentages) if they are not clearly implied by the JSON.

        =====================
        OUTPUT REQUIREMENTS
        =====================

        You MUST output in JSON format and follow this exact structure and HARD length caps:

        1) Section: "Key facts"
           - Output 3–5 bullet points.
           - Each bullet MUST be ≤ 18 words.
           - Focus on: what each row roughly represents, main entities, key columns, important datatypes (dates, geo, text), and granularity.
           - If counts (rows, columns) are missing or null, speak qualitatively (e.g., "many records") instead of making them up.
        - for column names, only output column names separated by commas, nothing else

        2) Section: "Use-cases"
           - Output 3–5 bullet points.
           - Each bullet MUST be ≤ 18 words.
           - Tailor to TOPIC and POTENTIAL_ANALYSES.
           - Include at least one analytical idea and one operational / business question.

        3) Section: "Caveats"
           - Output 3–5 bullet points.
           - Each bullet MUST be ≤ 18 words.
           - Mention schema limitations, missingness, lack of true coordinates, time coverage uncertainty, and any obvious biases from the columns.
           - If TIME_AND_GEO_COVERAGE is broader than what the schema suggests, flag that as a potential mismatch.

        4) Section: "Overview"
           - A single paragraph (NO bullets).
           - The paragraph MUST be between 40 and 80 words.
           - Summarize: what the dataset is about, likely time/geo scope (using TIME_AND_GEO_COVERAGE where appropriate), and typical analyses.
           - Mention TOPIC naturally, but do not restate all details from above bullet sections.

        =====================
        STYLE & CONSTRAINTS
        =====================

        - Do NOT exceed any of the word caps, even if you must omit information.
        - Do NOT mention internal JSON structure, keys, or field names like "dataset_signals" or "column_signals".
        - Do NOT say "the JSON says" or "the schema indicates"; write as a normal human-facing description.
        - Do NOT fabricate: if something is unknown or ambiguous from the JSON, either omit it or state it as uncertain.
        - Be clear and non-technical; assume an analytically literate but non-expert reader.
        - Output ONLY the four sections in this order, with these exact headings:

        Key facts
        Use-cases
        Caveats
        Overview

        """

    response = client.responses.create(
        model=model,
        input=user_prompt,
    )

    return response.output_text

In [10]:
profile = profile_dataframe(df)
signals = build_data_signals(profile)
summary = summarize_dataset_with_llm(profile, signals)
print(summary)

/tmp/ipython-input-977247162.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(sample, errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-977247162.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(sample, errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-977247162.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetim

```json
{
  "Key facts": [
    "Dataset contains many records related to parks and sculptures.",
    "Key columns include name, borough, parkname, and location.",
    "Dates and text fields dominate, with some numerical data.",
    "Geospatial information is present but lacks precise coordinates.",
    "Time coverage spans from the 1930s to 2021."
  ],
  "Use-cases": [
    "Analyze park distribution across boroughs over time.",
    "Examine correlations between materials used and sculpture types.",
    "Identify trends in park maintenance and funding.",
    "Assess public engagement through community boards.",
    "Explore historical significance of sculptures in different neighborhoods."
  ],
  "Caveats": [
    "High fraction of missing data in several columns.",
    "Not all geographic locations have true coordinates.",
    "Time coverage has some uncertainty regarding earlier records.",
    "Potential bias in sculpture representations across boroughs.",
    "Some columns may have ov